In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Fruit Freshness Index

This notebook attempts to determine a freshness index provided an image of a fruit.

**How the algorithm works?**

Suppose we are training a model to determine the freshness index of a banana.
1. We use a collection of rotten banana images.
2. These images are passed to an *EfficientNet* model and the features are extracted from the dataset.
4. Now we have a distribution which essentially represents the collection of rotten bananas.
5. Now we find the **mean** and **covariance matrix** of this distribution.
6. Suppose we have a test banana. We calculate the **Mahalanobis Distance** of the banana from the distribution. For more reference on Mahalanobis distance: https://www.machinelearningplus.com/statistics/mahalanobis-distance/
7. The more the Mahalanobis distance implies the furthur the test point is from the distribution, implies that the banana is more fresh.
8. That is how we classify the freshness index.

### **Subsequent improvements to the model:**

The model uses EfficientNet to extract the features. EfficientNet has been chosen as it is real time. Furthur it is a CNN architecture so it is expected that the image features extracted won't depend on the camera angle. The drawback is that the model draws all information including the background information. So some predictions are not that accurate. The workaround is to use Segmentation Techniques and Vision transformers to extract only what is necessary.

In [ ]:
# Importing torch and setting up device agnostic code
import torch

# Set up device agnostic code

# Check if GPU is available and send the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# Feature Extraction using EfficientNet

import torch
import torch.nn as nn
from torchvision import models

class EfficientNet_FeatureExtractor(nn.Module):

    def __init__(self):
        super(EfficientNet_FeatureExtractor, self).__init__()
        # Load a pre-trained EfficientNet model (e.g., EfficientNet-B0)
        self.efficientnet = models.efficientnet_b0(pretrained=True)

        # Remove the classifier (the last fully connected layer)
        self.efficientnet = nn.Sequential(*list(self.efficientnet.children())[:-1])

    def forward(self, x):
        # Forward pass through EfficientNet until before the classifier
        x = self.efficientnet(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        return x

# Example usage:
# model = EfficientNet_FeatureExtractor()
# features = model(input_tensor)

In [ ]:
model = EfficientNet_FeatureExtractor()
model

In [ ]:
# Calculating the mean and variance of the images whose features will be extracted

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# Define a transform to convert images to tensors without normalizing them
transform = transforms.Compose([
    transforms.Resize(256),        # Resize the shorter side to 256 pixels
    transforms.CenterCrop(224),    # Crop the center to get a 224x224 image
    transforms.ToTensor(),
])

# Load your dataset
# Assuming you have a folder containing your custom images (custom_dataset_folder)
dataset = datasets.ImageFolder(root='/kaggle/input/bananas-dataset/Dataset', transform=transform)

# Create a DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# Initialize variables to calculate the mean and std
mean = 0.0
std = 0.0
total_images = 0

# Iterate over the dataset to compute mean and std
for images, _ in loader:
    batch_samples = images.size(0)  # Number of images in the batch
    images = images.view(batch_samples, images.size(1), -1)  # Flatten each image (C, H*W)

    # Calculate mean and std for this batch and add to the running total
    mean += images.mean(2).sum(0)  # Mean across the H*W dimensions
    std += images.std(2).sum(0)    # Std across the H*W dimensions
    total_images += batch_samples

# Final mean and std across all images in the dataset
mean /= total_images
std /= total_images

print(f"Mean: {mean}")
print(f"Std: {std}")

In [ ]:
# Assuming you calculated mean and std as follows
# mean = [0.7245, 0.6862, 0.6531]  # Your calculated values
# std = [0.2138, 0.2454, 0.3047]   # Your calculated values

# Transforming the images into the format so that they can be passes through the EfficientNet model

# Define the transform for your dataset, including normalization with custom mean and std
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)  # Use your custom mean and std
])

# Load the dataset with the updated transform
test_dataset = datasets.ImageFolder(root='/kaggle/input/bananas-dataset/Dataset', transform=transform)


In [ ]:
test_dataset

In [ ]:
# Extracting features from Efficientnet model


# Initialize the feature extractor model
model = EfficientNet_FeatureExtractor().to(device)
model.eval()  # Set to evaluation mode

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)


# Store the extracted features
all_features = []

# Loop over the test dataset and extract features
with torch.no_grad():  # Disable gradient calculation for efficiency
    for images, _ in test_loader:
        # Send the images to the same device as the model
        images = images.to(device)

        # Pass the images through the feature extractor
        features = model(images)

        # Move features to CPU and convert to NumPy (optional)
        features = features.cpu().numpy()

        # Append the features for further use
        all_features.append(features)

# Print the shape of each batch stored in the list
for i, features in enumerate(all_features):
    print(f"Shape of batch {i}: {features.shape}")

In [ ]:
# Calculating the mean and varinance of the entire distribution

import torch

# Stack all the feature vectors into a single tensor
all_features_tensor = torch.cat([torch.tensor(batch) for batch in all_features], dim=0)

# Calculate the mean and variance along the feature dimension
# Feature dimension is typically the second axis (axis 1) in the feature vectors
feature_mean = all_features_tensor.mean(dim=0)
feature_mean = feature_mean.to(device)
feature_variance = all_features_tensor.var(dim=0)

print(f"Feature Mean Shape: {feature_mean.shape}")

In [ ]:
# Calculating the Covariance Matrix of the distribution

import torch

# Convert the list of arrays into one large tensor
all_features_tensor = torch.cat([torch.tensor(f) for f in all_features], dim=0)

# Move the stacked tensor to the device (if you're using GPU)
all_features_tensor = all_features_tensor.to(device)

# Calculate the mean along the batch dimension
feature_mean_temp = all_features_tensor.mean(dim=0)

# Center the feature vectors by subtracting the mean
centered_features = all_features_tensor - feature_mean_temp

# Calculate the covariance matrix
# Covariance matrix: (num_features, num_features)
covariance_matrix = torch.cov(centered_features.T)
covariance_matrix = covariance_matrix.to(device)

print(f"All Feature Tensor Shape: {all_features_tensor.shape}")
print(f"Covariance Matrix Shape: {covariance_matrix.shape}")

In [ ]:
# Defining the function to calculate the Mahalanobis distance

import torch

def mahalanobis(x=None, feature_mean=None, feature_cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data
    x             : tensor of shape [batch_size, num_features], feature vectors of test data
    feature_mean  : tensor of shape [num_features], mean of the training feature vectors
    feature_cov   : tensor of shape [num_features, num_features], covariance matrix of the training feature vectors
    """

    # Subtract the mean from x
    x_minus_mu = x - feature_mean

    # Invert the covariance matrix
    inv_covmat = torch.inverse(feature_cov)

    # Mahalanobis distance computation: (x - mu)^T * inv_cov * (x - mu)
    left_term = torch.matmul(x_minus_mu, inv_covmat)  # Shape: [batch_size, num_features]
    mahal = torch.matmul(left_term, x_minus_mu.T)     # Shape: [batch_size, batch_size]

    # Return the diagonal elements which are the distances for each sample
    return mahal.diag()

# Example usage:
# x, feature_mean, and feature_cov should all be PyTorch tensors
# x: shape [batch_size, num_features]
# feature_mean: shape [num_features]
# feature_cov: shape [num_features, num_features]

In [ ]:
# Testing on a single image

from PIL import Image
from torchvision import transforms

# Define the transform (same as before)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)  # Use your custom mean and std
])

# Load the image
img_path = '/kaggle/input/fresh-and-stale-classification/dataset/Train/rottenbanana/Screen Shot 2018-06-12 at 8.50.20 PM.png'  # Example image path
img = Image.open(img_path)

# Print image mode
print(f"Image mode: {img.mode}")

# If the image is in RGBA or another format, convert to RGB
if img.mode != 'RGB':
    img = img.convert('RGB')


# Apply the transformation
img_transformed = transform(img)

# If needed, add a batch dimension (because models expect batches)
img_transformed = img_transformed.unsqueeze(0)  # Shape: [1, 3, 224, 224]

# Move the transformed image to the appropriate device (GPU/CPU)
img_transformed = img_transformed.to(device)

# Ensure the model is on the same device
model = model.to(device)

# Set model to eval mode for feature extraction
model.eval()

# Pass the images through the feature extractor (no gradient needed)
with torch.no_grad():
    features_1 = model(img_transformed)

# Move the features to the same device as feature_mean and covariance_matrix (if needed)
features_1 = features_1.to(device)

# Ensure feature_mean and covariance_matrix are also on the correct device
feature_mean = feature_mean.to(device)
covariance_matrix = covariance_matrix.to(device)

# Calculate the Mahalanobis distance for the feature vector
distance = mahalanobis(features_1, feature_mean, covariance_matrix)
distance = torch.abs(distance) / 1e8

In [ ]:
def classify_banana_by_distance(distance):
    """
    Classifies the banana's freshness based on the Mahalanobis distance.

    Args:
        distance (float): Mahalanobis distance of the banana.

    Returns:
        dict: A dictionary containing the classification and relevant details.
    """

    # Define thresholds for classification based on the provided distances
    if distance >= 9:
        # Case 1: Completely Fresh Banana
        return {
            "Classification": "Completely Fresh",
            "Freshness Index": 10,
            "Color": "Mostly yellow, little to no brown spots",
            "Dark Spots": "0-10%",
            "Shelf Life": "5-7 days",
            "Ripeness Stage": "Just ripe",
            "Texture": "Firm and smooth"
        }
    elif 5 <= distance < 9:
        # Case 2: Banana with 40% Dark Brown Spots
        return {
            "Classification": "Moderately Ripe",
            "Freshness Index": 6,
            "Color": "60% yellow, 40% dark spots",
            "Dark Spots": "40% dark spots",
            "Shelf Life": "2-3 days",
            "Ripeness Stage": "Moderately ripe",
            "Texture": "Some softness, still edible"
        }
    else:
        # Case 3: Almost Rotten Banana
        return {
            "Classification": "Almost Rotten",
            "Freshness Index": 2,
            "Color": "Mostly brown or black, very few yellow patches",
            "Dark Spots": "80-100% dark spots",
            "Shelf Life": "0-1 days",
            "Ripeness Stage": "Overripe",
            "Texture": "Very soft, mushy, may leak moisture"
        }

results = classify_banana_by_distance(distance)